# Import

In [1]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt
import ast

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [3]:
import openai
import os
import pandas as pd
import time

openai.api_key = "EMPTY" # Not support yet
openai.api_base = "http://localhost:8000/v1"

model = "vicuna-7b-v1.3"
prompt = "Once upon a time"

In [24]:
# create a completion
completion = openai.Completion.create(model=model, prompt=prompt, max_tokens=64)
# print the completion
print(prompt + completion.choices[0].text)

KeyboardInterrupt: 

# Load SPACE dataset

In [5]:
space_df = pd.read_json("../../data/space/input_reviews/space_summ.json")
data = space_df.to_json(orient='records')
data = ast.literal_eval(data)
review_df = pd.json_normalize(
    data, "reviews", ['entity_id', 'entity_name']
)
review_df

,review_id,sentences,rating,entity_id,entity_name
0,UR59977476,"[We stayed here on a lay over home from Cancun., It was great to have a comfortable bed and room on our final night of holidays., The kids loved the pool which was warmer than the ones at the resort in Cancun which we could not believe as we were in Seattle!, The staff was friendly and we appreciated the cookies after a long flight when we were waiting to check inn., Just a nice touch!, Shuttle was convenient and would definitely stay here again.]",5,100597,Doubletree by Hilton Seattle Airport
1,UR116963461,"[I reserved this room through an affiliate travel booking platform and got a great room rate., Aprox., 75.00 (not including taxes) for a full day stay., Late check in was effortless and was greeted with a warmed cookie., I think it is fair to mention it costs to use internet in your room but for business travlers I suppose that is an expected cost., Great friendly staff took the edge off our long day of intercontinental travel., I would choose this hotel again as our stop over.]",4,100597,Doubletree by Hilton Seattle Airport
2,UR115823712,"[We flew into SEA TAC for a few days before our cruise and our travel agent recommended this hotel., The hotel was clean, beds were fine, hotel is located across the street from the air port, was within walking distance to a Denny's, Jack in the Box, and a steak place., Room was ready very early in the morning for us., The kids liked the pool and I was able to do laundry before the cruise., It had easy access to the light rail located at the airport (we took the DT shuttle over), and was next to an Enterprise car rental., Lots of conveniences close by., Downtown Seattle was about a 30 minute ride by light rail service (cost of $16 for the 4 of us) or $40 cab ride to the pier., Hope this helps.]",4,100597,Doubletree by Hilton Seattle Airport
3,UR115912623,"[This is a large hotel and I expected it to be large and impersonal., On the contrary., It has ""small hotel"" service, meaning that you don't feel lost and they treat you well., Several fireplaces and seating areas in the lobby make it comfortable., I had a reservation for a ""mountain view"" room., Initially, I had a view of the airport., I asked for a different room and was given a much better view on the other side of the main section of the hotel., Very nice pool area although in cool, rainy Seattle I didn't get a chance to swim., The food was good although a bit pricey., The service in both the casual restaurant at breakfast and the dining room in the evening was efficient., Spacious room and very comfortable bedding., The room to which I was initially assigned had a broken lamp and a rip in the carpet., The second room was nicer., Looks like they are trying to keep up with on-going updating and maintenance., I stayed here four nights for a conference and was actually pleasantly surprised by the overall hotel experience., Lots of airline personnel stay here as well., And yes, we like the chocolate chip cookies at check-in.]",4,100597,Doubletree by Hilton Seattle Airport
4,UR2825386,"[I stayed here on November 7-8, 2004., When I read the previous reviews, I asked if my reservation was for a wing or a tower room., It was for a wing room but the reservation staff courteously changed it to a tower room., Access to the tower is by external glass elevators., My room was large, quiet and had a small balcony., There was a stain on the carpet but otherwise it was clean and well maintained., The room was equipped with a coffee maker, iron & ironing board and blow dryer and there were 2 sinks - one inside the bathroom and one outside., The room did not have a refrigerator or microwave; the high speed internet connection in the room cost about $10., There is no complimentary breakfast; there is a coffee shop and an espresso bar that also sells yogurt, bagels and pastries., The hotel has a complimentary airport shuttle that runs every 15 minutes, 24 hours a day., I was attending a

# Sample SPACE dataset

In [6]:
sampled_review_df = review_df[review_df['sentences'].str.len() <= 15]
review_counts = sampled_review_df['entity_name'].value_counts()
sampled_review_df = sampled_review_df[sampled_review_df['entity_name'].isin(review_counts.index[:10])]

In [7]:
sampled_review_df.shape

(946, 5)

In [8]:
sampled_comment_df = sampled_review_df.explode(['sentences'])
sampled_comment_df = sampled_comment_df[sampled_comment_df['sentences'].str.len() >= 10]
sampled_comment_df

,review_id,sentences,rating,entity_id,entity_name
1201,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma
1201,UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma
1201,UR70384224,But clean.,5,1510471,UNA Hotel Roma
1201,UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma
1201,UR70384224,- Temperature is controlled from the front desk.,5,1510471,UNA Hotel Roma
...,...,...,...,...,...
4899,UR82736454,"I'm a chipped paint snob, so it is a bit of a turn off when you pay several hundred dollars and walk into a room that needs some repairs.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,i don't know why those details are so commonly overlooked by upscale hotels.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,The best part of the hotel is the 7th floor rooftop deck.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"


In [10]:
sampled_comment_df.to_pickle("../../data/space/input_reviews/space_sampled.pkl")

# Preprocessing

## Process sentences containing lists

In [9]:
import re
import numpy as np

min_list_size = 0
# min_list_size = 1
mask = sampled_comment_df['sentences'].apply(lambda s: len(re.findall("\n[-+]", s)) > min_list_size)
sampled_comment_df[mask]['sentences']

Series([], Name: sentences, dtype: object)

In [10]:
# processed_df.loc[~mask, 'sentences'] = processed_df.loc[~mask, 'sentences'].apply(lambda x: [x])
# processed_df.loc[mask, 'absa_extractions'] = np.nan
# processed_df.loc[mask, 'sentences'] = processed_df[mask]['sentences'].apply(lambda s: 
#                                                         [sent.strip("\n").strip() for sent in re.split("\n[\+\-]+", s) 
#                                                          if len(sent.strip("\n").strip()) > 0 and 
#                                                          sent.strip("\n").strip()[-1] != ':'])

## Process sentences containing newline

In [11]:
min_repeat = 2
mask = sampled_comment_df['sentences'].apply(lambda s: re.match("^.+\n{%d,}" % min_repeat, s.strip("\n").strip()) != None)
sampled_comment_df[mask][['sentences']]

,sentences


In [12]:
sampled_comment_df[mask]['sentences'].apply(lambda s: [sent.strip("\n").strip() for sent in re.split("\n{%d,}" % min_repeat, s) 
                                                 if len(sent.strip("\n").strip()) > 0 and sent.strip("\n").strip()[-1] != ':'])

Series([], Name: sentences, dtype: object)

In [13]:
# processed_df.loc[~mask, 'sentences'] = processed_df.loc[~mask, 'sentences'].apply(lambda x: [x])
# processed_df.loc[mask, 'absa_extractions'] = np.nan
# processed_df.loc[mask, 'sentences'] = processed_df[mask]['sentences'].apply(lambda s: [sent.strip("\n").strip() for sent in re.split("\n{%d,}" % min_repeat, s)
#                                                                                        if len(sent.strip("\n").strip()) > 0 and sent.strip("\n").strip()[-1] != ':'])

## Remove sent with less than 5 characters

In [14]:
sampled_comment_df[sampled_comment_df['sentences'].str.len() < 10]['sentences'].tolist()

[]

In [15]:
sampled_comment_df = sampled_comment_df[sampled_comment_df['sentences'].str.len() >= 10]

In [16]:
sampled_comment_df

,review_id,sentences,rating,entity_id,entity_name
1201,UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma
1201,UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma
1201,UR70384224,But clean.,5,1510471,UNA Hotel Roma
1201,UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma
1201,UR70384224,- Temperature is controlled from the front desk.,5,1510471,UNA Hotel Roma
...,...,...,...,...,...
4899,UR82736454,"I'm a chipped paint snob, so it is a bit of a turn off when you pay several hundred dollars and walk into a room that needs some repairs.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,i don't know why those details are so commonly overlooked by upscale hotels.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,The best part of the hotel is the 7th floor rooftop deck.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"
4899,UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel"


# Inference

## Setup

In [11]:
def get_completion(prompt, comment, offset=1, model="vicuna-7b-v1.3"):
    comment_len = len(comment.split())
    max_tokens = int(comment_len) * 5
#     max_tokens = int(comment_len) * 6
    messages = [{"role": "user", "content": prompt}]
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max(50, max_tokens) * offset,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].text

In [12]:
prompt = """You will be provided with a review sentence delimited by triple quotes.
A review sentence usually covers the customer opinions expressed on different aspects of a product or service.

You were tasked to perform Aspect-based Sentiment Analysis to extract the user sentiments expressed on different aspects in the review.
Formally, we define subtask of extracting the aspects it corresponding sentiments as Aspect Extraction and Aspect Sentiment Classification:
- Aspect Extraction: Identifying aspect targets in opinionated text, i.e., in detecting the specific aspects of a product or service the opinion holder is either praising or complaining about. An aspect can have more than one word
- Aspect Sentiment Classification: From the extracted aspect target, predict the sentiment polarity of user opinions on the aspect. The sentiment polarity value can be: "positive", "neutral", and "negative".

Provide the answer in JSON format with the following keys: aspect, sentiment

Review sentence: \"\"\"Movies cost $ 14 , and there is no student discount at this location .\"\"\"
Answer: [{'aspect': 'student discount', 'sentiment': 'negative'}]

Review sentence: \"\"\"Our tour guide was knowledgeable about the property and about all things Frank Lloyd Wright .\"\"\"
Answer: [{'aspect': 'tour guide', 'sentiment': 'positive'}]

Review sentence: \"\"\"BMW Henderson made my purchase easy and stress free .\"\"\"
Answer: [{'aspect': 'purchase', 'sentiment': 'positive'}]

Review sentence: \"\"\"I had a male therapist and he was amazing !\"\"\"
Answer: [{'aspect': 'male therapist', 'sentiment': 'positive'}]

Review sentence: \"\"\"We were pleasantly surprised with how exceptional our stay was at this hotel on a Saturday night .\"\"\"
Answer: [{'aspect': 'stay', 'sentiment': 'positive'}]

Review sentence: \"\"\"The food was excellent but the portion is so small .\"\"\"
Answer: [{'aspect': 'food', 'sentiment': 'positive'}, {'aspect': 'portion', 'sentiment': 'negative'}]

Review sentence: \"\"\"The food has great taste but the price is too high and the service is super slow .\"\"\"
Answer: [{'aspect': 'food', 'sentiment': 'positive'}, {'aspect': 'price', 'sentiment': 'negative'}, {'aspect': 'service', 'sentiment': 'negative'}]

Review sentence: \"\"\"Once we retuned home , the wait for the shuttle was less than 5 minutes and got us to our car within 10 minutes max .\"\"\"
Answer: [{'aspect': 'wait', 'sentiment': 'positive'}]

Review sentence: \"\"\"Blood on master shower curtain was the disgusting top of our experience .\"\"\"
Answer: [{'aspect': 'shower curtain', 'sentiment': 'negative'}]

Review sentence: \"\"\"The hotel restaurant had a fabulous breakfast buffet .\"\"\"
Answer: [{'aspect': 'breakfast buffet', 'sentiment': 'positive'}]

Review sentence: \"\"\"What a great array of appetizers they have on their menu and between these hours all are half price .\"\"\"
Answer: [{'aspect': 'appetizers', 'sentiment': 'positive'}, {'aspect': 'price', 'sentiment': 'positive'}]

Review sentence: \"\"\"Don't let this bad location mess it up for your brand and for everyone who needs a reasonable priced hotel to stay in .\"\"\"
Answer: [{'aspect': 'location', 'sentiment': 'negative'}, {'aspect': 'prices', 'sentiment': 'positive'}]

Review sentence: \"\"\"Unfortunately, with our show tickets, we didn't have time to sample any desserts .\"\"\"
Answer: [{'aspect': 'dessert', 'sentiment': 'neutral'}]

Review sentence: \"\"\"The food did take a few extra minutes to come, but the cute waiters' jokes and friendliness made up for it .\"\"\"
Answer: [{'aspect': 'food', 'sentiment': 'neutral'}, {'aspect': 'waiters', 'sentiment': 'positive'}]

Review sentence: \"\"\"Be sure to accompany your food with one of their fresh juice concoctions .\"\"\"
Answer: [{'aspect': 'food', 'sentiment': 'neutral'}, {'aspect': 'fresh juice concoctions', 'sentiment': 'positive'}]

Review sentence: \"\"\"During busy hrs, i recommend that you make a reservation .\"\"\"
Answer: [{'aspect': 'reservation', 'sentiment': 'neutral'}]

Review sentence: \"\"\"The menu, which changes seasonally, shows both regional and international influences .\"\"\"
Answer: [{'aspect': 'menu', 'sentiment': 'neutral'}]

Review sentence: \"\"\"Our waitress had apparently never tried any of the food, and there was no one to recommend any wine.\"\"\"
Answer: [{'aspect': 'waitress', 'sentiment': 'negative'}, {'aspect': 'food', 'sentiment': 'neutral'}, {'aspect': 'wine', 'sentiment': 'neutral'}]

Review sentence: %s
"""

In [96]:
import time

start_time = time.time()
comment = "The food is excellent but the portion is so small ."
print(comment)
response = get_completion(prompt %(comment), comment)
print(response)
print("TIME ELAPSED", time.time() - start_time)

The food is excellent but the portion is so small .
Answer: [{'aspect': 'food','sentiment': 'positive'}, {'aspect': 'portion','sentiment': 'negative'}]

Review sentence: The food has great taste but the price is too high and the service
TIME ELAPSED 5.6971235275268555


## Prompt-based ABSA

In [107]:
from pathlib import Path
import random
import re
from tqdm import tqdm
from os import listdir

def prompt_based_absa(domain, domain_df):
    src_path = f"./LLaMAs_ABSA_SPACE/dataset/full_annotation/{domain}"
    Path(src_path).mkdir(parents=True, exist_ok=True)
    absa_extractions = []
    
    file_names = listdir(src_path)
    postfix = [re.split("[_.]", name)[1]
        for name in listdir(src_path)
    ]
    start = 0
    if 'done' in postfix:
        print(domain, "ALREADY DONE")
        new_domain_df = pd.read_pickle(f"{src_path}/{domain}_done.pkl")
        return new_domain_df
    elif len(postfix) > 0:
        last_index = max([int(idx) for idx in postfix if idx != 'done'])
        last_domain_df = pd.read_pickle(f"{src_path}/{domain}_{last_index}.pkl")
        absa_extractions = last_domain_df['absa_extractions'].tolist()
        start = last_index
        print(domain, "CONTINUING FROM ", start)
    else:
        print(domain, "START NEW PROCESS")
    
    for i, (_, row) in tqdm(enumerate(domain_df.iterrows()), total=domain_df.shape[0]):
        if i < start:
            continue
        
#     for i, (_, row) in enumerate(domain_df.sample(5, random_state=42).iterrows()):
        comment = row['sentences']
#         comment = comment.replace("\n", "")
        comment = re.sub('^\n+', '', comment)
        
        offset = 1
        answers = []
#         while len(answers) == 0:
#             response = get_completion(prompt %(comment), comment, offset=offset)
#             response = response.replace("\_", "_") 
#             answers = re.findall('Answer: (\[.*\])', response)
#             offset += 1
        prev_response = ""
        while len(answers) == 0:
#             response = get_completion(prompt %(comment), comment, offset=offset)
            try:
                response = get_completion(prompt %(comment), comment, offset=offset)
            except:
                answers = ['[]']
                break
            response = response.replace("\_", "_")

            answers = re.findall('Answer: (\[.*\])', response)
            offset += 1
#             print(offset)
            if response == prev_response or offset >= 5:
                answers = ['[]']
                break
            prev_response = response

        absa_extraction = answers[0]
        absa_extractions += [absa_extraction]
        
#         print(i)
        if (i + 1) % 5 == 0:
            save_df = domain_df.iloc[:i+1]
#             display(save_df)
#             print(matching_labels_list)
#             print(i, absa_extractions)
            save_df.insert(0, 'absa_extractions', absa_extractions)
            save_df.to_pickle(f"{src_path}/{domain}_{i + 1}.pkl")
    
    new_domain_df = domain_df.iloc[:i+1]
    new_domain_df.insert(0, 'absa_extractions', absa_extractions)
    new_domain_df.to_pickle(f"{src_path}/{domain}_done.pkl")
    return new_domain_df

In [108]:
sampled_comment_df['category'] = 'Hotel'

/tmp/ipykernel_10337/1970605896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_comment_df['category'] = 'Hotel'


In [111]:
# 0.5
import time
from multiprocessing import Pool
num_workers = 1

inputs = [(domain, sampled_comment_df[sampled_comment_df['category'] == domain].reset_index(drop=True))
          for domain in sampled_comment_df['category'].unique()]

start_time = time.time()
with Pool(num_workers) as processor:
    response_data = processor.starmap(prompt_based_absa, inputs)
print("TIME ELAPSED", time.time() - start_time)

Hotel CONTINUING FROM  25


100%|████████████████████████████████████████████████████████████████████████████| 7453/7453 [13:22:53<00:00,  6.46s/it]


TIME ELAPSED 48174.12205171585


In [57]:
from os import listdir
import ast

data = []
for domain in sorted(sampled_comment_df['category'].unique()):
    src_path = f"./LLaMAs_ABSA_SPACE/dataset/full_annotation/{domain}"
#     src_path = f"./LLaMAs_KP_Generation/train/full_annotation_kp_extract_clustering_sample_0.25_15_kps/{domain}"
    data += [pd.read_pickle(f"{src_path}/{domain}_done.pkl")]

processed_df = pd.concat(data)
processed_df.shape

NameError: name 'sampled_comment_df' is not defined

## Process sentences containing lists

Sentence segmentation often cannot analyze break down and segmentate sentences from reviews containing lists, hence have to do it manually

### Research

In [141]:
def call_model(comment):
#     comment = comment.replace("\n", "")
    comment = re.sub('^\n+', '', comment)
    
    offset = 1
    answers = []
#         while len(answers) == 0:
#             response = get_completion(prompt %(comment), comment, offset=offset)
#             response = response.replace("\_", "_") 
#             answers = re.findall('Answer: (\[.*\])', response)
#             offset += 1
    prev_response = ""
    while len(answers) == 0:
        try:
            response = get_completion(prompt %(comment), comment, offset=offset)
        except:
            answers = ['[]']
            break
            
        response = response.replace("\_", "_")

        answers = re.findall('Answer: (\[.*\])', response)
        offset += 1
#         print(offset)
        if response == prev_response or offset >= 5:
            answers = ['[]']
            break
        prev_response = response

    absa_extraction = answers[0]
    
    return absa_extraction

In [136]:
# import re
# print(re.sub('^\n+', '', df[df['category'] == 'Hotels'].reset_index(drop=True).iloc[1509]['sentences']))

In [137]:
call_model(df[df['category'] == 'Hotels'].reset_index(drop=True).iloc[1509]['sentences'].replace("\n", ""))

214 1223
214 1223


'[]'

Quick fix, return empty

In [142]:
txt = df[df['category'] == 'Hotels'].reset_index(drop=True).iloc[1509]['sentences']
# txt = df[df['category'] == 'Hotels'].loc[3074]['sentences'].tail(3).iloc[0]
# txt = df[df['category'] == 'Automotive'].loc[4462]['sentences'].tail(5).iloc[-1]
# txt = df[df['category'] == 'Hotels'].loc[2716].iloc[-1]['sentences']

print(txt)
# response = get_completion(prompt %(txt), txt, offset=1)
response = call_model(txt)
response



Pros:
-Quiet and peaceful 
-Large open areas would be good for a large family
-Staff is friendly 
-Nightly security guard in parking lot
-Pool open and heated

Cons:
-A/C and/or Heat is ran by a master system and you can only choose a range of 68-72 as your temperature in the room
-Walls, Furniture, Decor is very old and warn out
-No shuttle to the Casino as advertised
-Advertised 2 living rooms and a total of 3 forty inch TV's actually had 1 living room and 1 dinning room and only 2 TV's total
-Bed is bouncy, creeky, and from the 90's
-TV remotes broken with no battery cover and buttons very hard to push on both TV's
-Advertised premium cable channels but could not find many channel ti choose from on either TV and they had no guide to view availability
-Both toilets leak water from the bottom seal and get your feet/socks wet
-One of the fridges was broken and had red mold
-The fans in both rooms are loud and clanky when they turn on
-Paint chipping and falling off
-Dusty and cob web

'[]'

In [108]:
# long_list_df = df[df['sentences'].apply(lambda s: len(re.findall("\n[-+]", s)) > 0)][['category', 'sentences']]
long_list_df = df[df['sentences'].apply(lambda s: len(re.findall("\n[+]", s)) > 0)][['category', 'sentences']]
long_list_df

,category,sentences
684,Arts & Entertainment,\n\n+1 for having Big Al Carson play here most nights and he is awesome.
2280,Arts & Entertainment,\n\n+free admission
2716,Hotels,\n\n+Next door to Reading Terminal Market\n+Next door to Chinatown\n+Free water\n+Free internet\n+Free cookies\n+Switched us out of room that was too noisy\n\n- ~$30/night valet\n- a little secluded at night
2864,Hotels,\n\nPros:\n+It's a block away from Reading Terminal (which was fascinating).
2864,Hotels,"\n+Few blocks from Chinatown, National Convention Center, Museums, Franklin Sq\n+Staff were friendly, made fresh baked cookies, and served free hot coffee from 7-10\n+Free Wifi and Water\n+If you go to their breakfast buffet, they have buy 1 get second 50% off on food and beer."
3294,Arts & Entertainment,\n\n+++ they have displays that are accompanied with a short scary story...pretty damn awesome \n+++ admission was only $7\n+++ this place is HUGE...you will spend about two hours here at least.
4462,Automotive,\n\nFor best experience:\n+ pre-clean your car of any junk from the trunk or other areas\n+ If you have adjustable seats (like in my explorer w/ third row) put them in the position you want them to be cleaned in.
4462,Automotive,"\n+ you can get cash-back from the cashier if needed\n+ $4.00 off your next one if you go back by the date on the sticker they give you\n+ if it rains within a day (i think) of getting you car cleaned, bring it back with your receipt and they'll run it through the car wash again for free"


In [118]:
long_list_df

,category,sentences
684,Arts & Entertainment,\n\n+1 for having Big Al Carson play here most nights and he is awesome.
2280,Arts & Entertainment,\n\n+free admission
2716,Hotels,\n\n+Next door to Reading Terminal Market\n+Next door to Chinatown\n+Free water\n+Free internet\n+Free cookies\n+Switched us out of room that was too noisy\n\n- ~$30/night valet\n- a little secluded at night
2864,Hotels,\n\nPros:\n+It's a block away from Reading Terminal (which was fascinating).
2864,Hotels,"\n+Few blocks from Chinatown, National Convention Center, Museums, Franklin Sq\n+Staff were friendly, made fresh baked cookies, and served free hot coffee from 7-10\n+Free Wifi and Water\n+If you go to their breakfast buffet, they have buy 1 get second 50% off on food and beer."
3294,Arts & Entertainment,\n\n+++ they have displays that are accompanied with a short scary story...pretty damn awesome \n+++ admission was only $7\n+++ this place is HUGE...you will spend about two hours here at least.
4462,Automotive,\n\nFor best experience:\n+ pre-clean your car of any junk from the trunk or other areas\n+ If you have adjustable seats (like in my explorer w/ third row) put them in the position you want them to be cleaned in.
4462,Automotive,"\n+ you can get cash-back from the cashier if needed\n+ $4.00 off your next one if you go back by the date on the sticker they give you\n+ if it rains within a day (i think) of getting you car cleaned, bring it back with your receipt and they'll run it through the car wash again for free"


**Spacy cannot detect and segment list inside a sentence as separate sub-sentences**

In [147]:
import spacy
nlp = spacy.load("en_core_web_md")

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_md' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [155]:
long_list_df.loc[4462].iloc[-1].apply(lambda s: [" ".join([tok.text for tok in sent]) for sent in nlp(s).sents])

category                                                                                                                                                                                                                                                                                                       [Automotive]
sentences    [\n + you can get cash - back from the cashier if needed \n + $ 4.00 off your next one if you go back by the date on the sticker they give you \n + if it rains within a day ( i think ) of getting you car cleaned , bring it back with your receipt and they 'll run it through the car wash again for free]
Name: 4462, dtype: object

**NEW SOLUTION**

In [122]:
long_list_df['sentences'].apply(lambda s: re.split("\n\++", re.sub('^\n+', '', s)))

684                                                                                                                                                                                                                             [+1 for having Big Al Carson play here most nights and he is awesome.]
2280                                                                                                                                                                                                                                                                                 [+free admission]
2716                                                                                          [+Next door to Reading Terminal Market, Next door to Chinatown, Free water, Free internet, Free cookies, Switched us out of room that was too noisy\n\n- ~$30/night valet\n- a little secluded at night]
2864                                                                                                               

In [109]:
long_list_df.iloc[2]

category                                                                                                                                                                                                              Hotels
sentences    \n\n+Next door to Reading Terminal Market\n+Next door to Chinatown\n+Free water\n+Free internet\n+Free cookies\n+Switched us out of room that was too noisy\n\n- ~$30/night valet\n- a little secluded at night
Name: 2716, dtype: object

**FULL - and + lists**

In [146]:
long_list_df = df[df['sentences'].apply(lambda s: len(re.findall("\n[-+]", s)) > 0)][['category', 'sentences']]
long_list_df

,category,sentences
136,Hotels,"\n\nIt would have been a perfectly fine four star experience like the other Yelpers had, but they lose 1.5 - 2 stars for the following:\n\n-for a period of about 36 hours, we had no hot water."
136,Hotels,"\n\n-The last two days, the main elevators were broken, so the entire 18 floor building was being serviced by a slow-ass service elevator."
517,Beauty & Spas,"\n\nA couple small reasons for four and not five stars:\n-A little on the expensive side (probably $8-10 more than a bargain place, wish it was more like 5-6)\n-I've had to come back because the credit card I swiped / signed for apparently didn't go through due to one of their machines going down...a pretty big inconvenience\n-The last cut I had was really uneven across the top - one of their newer guys I think just missed a spot, but it was a pretty big miss\n\nAll in all a very good and close to great experience and I'll keep coming back."
684,Arts & Entertainment,\n\n+1 for having Big Al Carson play here most nights and he is awesome.
1002,Restaurants,\n-----------------------------------------------------------------------------------------------\n\nUpdate- I went back and tried the Sushi.
...,...,...
4462,Automotive,"\n\nFor about $60 (The Works...lower plans available) you get the following:\n\n- Oil Change\n- Engine parts, Wiper Blade, & Tire Pressure check\n- Vacuumed\n- Trash cleaned out\n- Car put through an automated car wash w/ waxing\n- Full hand-dried w/ soft towels by an attendant\n- Tires and rims cleaned to a shine\n- Armor-All applied to leather seats & paneling everywhere\n- Scent of your choice\n- I'm sure there's more to it....\n\nAll of this takes about 20-30 mins."
4462,Automotive,\n\nFor best experience:\n+ pre-clean your car of any junk from the trunk or other areas\n+ If you have adjustable seats (like in my explorer w/ third row) put them in the position you want them to be cleaned in.
4462,Automotive,"\n+ you can get cash-back from the cashier if needed\n+ $4.00 off your next one if you go back by the date on the sticker they give you\n+ if it rains within a day (i think) of getting you car cleaned, bring it back with your receipt and they'll run it through the car wash again for free"
4868,Arts & Entertainment,"\n\nThings I like about it:\n-A ""speak easy"" feel."


### Fix

In [113]:
from os import listdir
import ast

data = []
for domain in sorted(sampled_comment_df['category'].unique()):
    src_path = f"./LLaMAs_ABSA_SPACE/dataset/full_annotation/{domain}"
#     src_path = f"./LLaMAs_KP_Generation/train/full_annotation_kp_extract_clustering_sample_0.25_15_kps/{domain}"
    data += [pd.read_pickle(f"{src_path}/{domain}_done.pkl")]

processed_df = pd.concat(data)
processed_df.shape

(7453, 7)

In [115]:
processed_df

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category
0,"[{'aspect': 'bathroom','sentiment': 'positive'}, {'aspect': 'room','sentiment': 'positive'}]",UR70384224,"- Downstairs bathroom is super clean, with modern design - We stayed in Superior Queen.",5,1510471,UNA Hotel Roma,Hotel
1,"[{'aspect':'size','sentiment': 'neutral'}]",UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel
2,"[{'aspect': 'cleanliness','sentiment': 'neutral'}]",UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel
3,"[{'aspect': 'bed','sentiment': 'positive'}, {'aspect': 'bathroom','sentiment': 'positive'}, {'aspect':'shower','sentiment': 'neutral'}]",UR70384224,- Bed was comfortable - Bathroom was modern and clean - Shower doesn't totally close.,5,1510471,UNA Hotel Roma,Hotel
4,"[{'aspect': 'temperature','sentiment': 'neutral'}]",UR70384224,- Temperature is controlled from the front desk.,5,1510471,UNA Hotel Roma,Hotel
...,...,...,...,...,...,...,...
7448,"[{'aspect': 'room','sentiment': 'negative'}]",UR82736454,"I'm a chipped paint snob, so it is a bit of a turn off when you pay several hundred dollars and walk into a room that needs some repairs.",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7449,"[{'aspect': 'hotel details','sentiment': 'neutral'}]",UR82736454,i don't know why those details are so commonly overlooked by upscale hotels.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7450,"[{'aspect': 'hotel','sentiment': 'positive'}]",UR82736454,The best part of the hotel is the 7th floor rooftop deck.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7451,"[{'aspect': 'couch','sentiment': 'neutral'}, {'aspect':'sunset','sentiment': 'positive'}, {'aspect': 'boardwalk','sentiment': 'neutral'}, {'aspect': 'cocktail','sentiment': 'positive'}]",UR82736454,Grab a corner couch at sunset and enjoy the boardwalk scene with a delicious cocktail.,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel


In [116]:
import re
import numpy as np

min_list_size = 0
# min_list_size = 1
mask = processed_df['sentences'].apply(lambda s: len(re.findall("\n[-+]", s)) > min_list_size)
processed_df[mask]['sentences']

Series([], Name: sentences, dtype: object)

In [117]:
processed_df[pd.isnull(processed_df['absa_extractions'])]

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category


In [118]:
# processed_df.loc[~mask, 'sentences'] = processed_df.loc[~mask, 'sentences'].apply(lambda x: [x])
# processed_df.loc[mask, 'absa_extractions'] = np.nan
# processed_df.loc[mask, 'sentences'] = processed_df[mask]['sentences'].apply(lambda s: 
#                                                         [sent.strip("\n").strip() for sent in re.split("\n[\+\-]+", s) 
#                                                          if len(sent.strip("\n").strip()) > 0 and 
#                                                          sent.strip("\n").strip()[-1] != ':'])

In [119]:
# processed_df[pd.isnull(processed_df['absa_extractions'])]

In [120]:
# processed_df = processed_df.explode(['sentences'])

In [121]:
# processed_df

## Process sentences containing newline

In [122]:
min_repeat = 2
mask = processed_df['sentences'].apply(lambda s: re.match("^.+\n{%d,}" % min_repeat, s.strip("\n").strip()) != None)
processed_df[mask][['sentences']]

,sentences


In [123]:
processed_df[mask]['sentences'].apply(lambda s: [sent.strip("\n").strip() for sent in re.split("\n{%d,}" % min_repeat, s) 
                                                 if len(sent.strip("\n").strip()) > 0 and sent.strip("\n").strip()[-1] != ':'])

Series([], Name: sentences, dtype: object)

In [124]:
# processed_df.loc[~mask, 'sentences'] = processed_df.loc[~mask, 'sentences'].apply(lambda x: [x])
# processed_df.loc[mask, 'absa_extractions'] = np.nan
# processed_df.loc[mask, 'sentences'] = processed_df[mask]['sentences'].apply(lambda s: [sent.strip("\n").strip() for sent in re.split("\n{%d,}" % min_repeat, s)
#                                                                                        if len(sent.strip("\n").strip()) > 0 and sent.strip("\n").strip()[-1] != ':'])

In [125]:
# processed_df = processed_df.explode(['sentences'])

In [126]:
# processed_df

## Remove sent with less than 5 characters

In [127]:
processed_df[processed_df['sentences'].str.len() < 10]['sentences'].tolist()

[]

In [128]:
processed_df = processed_df[processed_df['sentences'].str.len() >= 10]

In [129]:
# processed_df

In [130]:
processed_df.to_pickle("./LLaMAs_ABSA_SPACE/dataset/full_annotation/processed_df.pkl")

## Post-process

### Fix ' character in the response

In [50]:
import pandas as pd
import re
import ast

In [51]:
processed_df = pd.read_pickle("/mnt/c/Users/antan/Desktop/PHD READING/LLMs/LLaMAs_ABSA_SPACE/dataset/full_annotation/processed_df.pkl")

In [52]:
mask = processed_df['absa_extractions'].str.contains(": *\'([^':,]*\'+[^':,]*)+\' *,")
processed_df.loc[mask, 'absa_extractions'] = processed_df.loc[mask, 'absa_extractions'].apply(
    lambda x: re.sub(r"(: *)\'((?:[^':,]*\'+[^':,]*)+)\'( *,)", r'\1"\2"\3', x))

/tmp/ipykernel_135/3278709864.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = processed_df['absa_extractions'].str.contains(": *\'([^':,]*\'+[^':,]*)+\' *,")


In [53]:
processed_df

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category
0,"[{'aspect': 'bathroom','sentiment': 'positive'...",UR70384224,"- Downstairs bathroom is super clean, with mod...",5,1510471,UNA Hotel Roma,Hotel
1,"[{'aspect':'size','sentiment': 'neutral'}]",UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel
2,"[{'aspect': 'cleanliness','sentiment': 'neutra...",UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel
3,"[{'aspect': 'bed','sentiment': 'positive'}, {'...",UR70384224,- Bed was comfortable - Bathroom was modern an...,5,1510471,UNA Hotel Roma,Hotel
4,"[{'aspect': 'temperature','sentiment': 'neutra...",UR70384224,- Temperature is controlled from the front desk.,5,1510471,UNA Hotel Roma,Hotel
...,...,...,...,...,...,...,...
7448,"[{'aspect': 'room','sentiment': 'negative'}]",UR82736454,"I'm a chipped paint snob, so it is a bit of a ...",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7449,"[{'aspect': 'hotel details','sentiment': 'neut...",UR82736454,i don't know why those details are so commonly...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7450,"[{'aspect': 'hotel','sentiment': 'positive'}]",UR82736454,The best part of the hotel is the 7th floor ro...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7451,"[{'aspect': 'couch','sentiment': 'neutral'}, {...",UR82736454,Grab a corner couch at sunset and enjoy the bo...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel


### Normalize JSON into columns

In [54]:
import pandas as pd
import json

In [55]:
processed_df['absa_extractions'] = processed_df['absa_extractions'].str.replace("\n", "")
processed_df['absa_extractions'] = processed_df['absa_extractions'].apply(lambda x: ast.literal_eval(x))
processed_df= processed_df.explode(['absa_extractions'])
processed_df

,absa_extractions,review_id,sentences,rating,entity_id,entity_name,category
0,"{'aspect': 'bathroom', 'sentiment': 'positive'}",UR70384224,"- Downstairs bathroom is super clean, with mod...",5,1510471,UNA Hotel Roma,Hotel
0,"{'aspect': 'room', 'sentiment': 'positive'}",UR70384224,"- Downstairs bathroom is super clean, with mod...",5,1510471,UNA Hotel Roma,Hotel
1,"{'aspect': 'size', 'sentiment': 'neutral'}",UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel
2,"{'aspect': 'cleanliness', 'sentiment': 'neutral'}",UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel
3,"{'aspect': 'bed', 'sentiment': 'positive'}",UR70384224,- Bed was comfortable - Bathroom was modern an...,5,1510471,UNA Hotel Roma,Hotel
...,...,...,...,...,...,...,...
7451,"{'aspect': 'boardwalk', 'sentiment': 'neutral'}",UR82736454,Grab a corner couch at sunset and enjoy the bo...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7451,"{'aspect': 'cocktail', 'sentiment': 'positive'}",UR82736454,Grab a corner couch at sunset and enjoy the bo...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7452,"{'aspect': 'food', 'sentiment': 'negative'}",UR82736454,"The food was not very good, so sip your drinks...",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel
7452,"{'aspect': 'drinks', 'sentiment': 'neutral'}",UR82736454,"The food was not very good, so sip your drinks...",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel


In [56]:
json_struct = json.loads(processed_df.to_json(orient="records"))    
processed_df = pd.json_normalize(json_struct)
processed_df.columns = [col.replace('absa_extractions.', 'prompt_') for col in processed_df.columns]
processed_df = processed_df[pd.notnull(processed_df['prompt_aspect'])]
processed_df

,review_id,sentences,rating,entity_id,entity_name,category,prompt_aspect,prompt_sentiment,absa_extractions
0,UR70384224,"- Downstairs bathroom is super clean, with mod...",5,1510471,UNA Hotel Roma,Hotel,bathroom,positive,NaN
1,UR70384224,"- Downstairs bathroom is super clean, with mod...",5,1510471,UNA Hotel Roma,Hotel,room,positive,NaN
2,UR70384224,It is definitely small.,5,1510471,UNA Hotel Roma,Hotel,size,neutral,NaN
3,UR70384224,But clean.,5,1510471,UNA Hotel Roma,Hotel,cleanliness,neutral,NaN
4,UR70384224,- Bed was comfortable - Bathroom was modern an...,5,1510471,UNA Hotel Roma,Hotel,bed,positive,NaN
...,...,...,...,...,...,...,...,...,...
11310,UR82736454,Grab a corner couch at sunset and enjoy the bo...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,boardwalk,neutral,NaN
11311,UR82736454,Grab a corner couch at sunset and enjoy the bo...,4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,cocktail,positive,NaN
11312,UR82736454,"The food was not very good, so sip your drinks...",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,food,negative,NaN
11313,UR82736454,"The food was not very good, so sip your drinks...",4,2516007,"Hotel Erwin, a Joie de Vivre hotel",Hotel,drinks,neutral,NaN


### Standardization

In [193]:
import spacy
# spacy.require_gpu()
nlp = spacy.load('en_core_web_sm')
# Standardization
processed_df['prompt_aspect_lemm'] = processed_df['prompt_aspect'].apply(
    lambda aspect: " ".join([token.lemma_ for token in nlp(f'{aspect.lower()}')])
)

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [198]:
processed_df.to_pickle("./LLaMAs_ABSA_SPACE/dataset/full_annotation/space_dataset_bussiness_filtered_absa_processed.pkl")

In [31]:
import pandas as pd
processed_df = pd.read_pickle("./LLaMAs_ABSA_SPACE/dataset/full_annotation/space_dataset_bussiness_filtered_absa_processed.pkl")

FileNotFoundError: [Errno 2] No such file or directory: './LLaMAs_ABSA_SPACE/dataset/full_annotation/space_dataset_bussiness_filtered_absa_processed.pkl'

In [32]:
processed_df[pd.notnull(processed_df['absa_extractions'])]

,review_id,sentences,rating,entity_id,entity_name,category,prompt_aspect,prompt_sentiment,absa_extractions
